In [1]:
from copy import deepcopy

In [2]:
TEST_PATH = "test_input_8.txt"
PATH = "input_8.txt"

In [33]:
class InfiniteLoopEntered(Exception):
    def __init__(self, idx, accumulator):
        self.idx = idx
        self.accumulator = accumulator
        super().__init__(f"Infinite loop entered on index {idx}; accumulator value: {accumulator}")

class Operator:
    def __init__(self, file_name):
        self.operations = self._read_data(file_name)
        
        self._next_idx = 0
        self._executed_idxs = set()
        
        self.op_dict = {"acc": self.accumulate,
                        "jmp": self.jump,
                        "nop": self.no_operation}
        
        self.accumulator = 0
        
    def _read_data(self, file_name):
        with open(file_name, "r") as f:
            data = [(op, int(arg)) 
                    for idx, line in enumerate(f) 
                    for op, arg in [line[:-1].split() if line.endswith("\n") 
                                    else line.split()]]

        return data
    
    def __iter__(self):
        return self
    
    def __next__(self):        
        if self._next_idx in self._executed_idxs:
            raise InfiniteLoopEntered(self._next_idx, self.accumulator)
        
        try:
            op = self.operations[self._next_idx]    
        except IndexError:
            raise StopIteration()
        
        self._executed_idxs.add(self._next_idx)
            
        self.operation_factory(*op)
        
        return op, self.accumulator
    
    def operation_factory(self, op_type, val):
        self.op_dict[op_type](val)
        
    def accumulate(self, val):
        self.accumulator += val
        self._next_idx += 1

    def jump(self, val):
        self._next_idx += val

    def no_operation(self, val): 
        self._next_idx += 1
            
    def __reset(self):
        self.accumulator = 0
        self._next_idx = 0
        self._executed_idxs = set()
        
    def check_if_looped(self):
        while True:
            try:
                next(self)
            except InfiniteLoopEntered:
                self.__reset()
                return True
            except StopIteration:
                return False
    
    def _patch_operation(self, idx):
        swap_dict = {"jmp": "nop", "nop": "jmp"}
        op_type, op_val = self.operations[idx]
        if op_type not in swap_dict:
            return None
        prev_op = deepcopy(self.operations[idx])
        self.operations[idx] = (swap_dict[op_type], op_val)
        return prev_op, self.operations[idx]
        
    def patch_self(self):
        for idx in range(len(self.operations)):
            try:
                prev, op = self._patch_operation(idx)
            except TypeError:
                continue
                
            if self.check_if_looped():
                self._patch_operation(idx)
                continue
                
            return idx, prev, op

In [37]:
op = Operator(PATH)

print(op.check_if_looped())
op.patch_self()

try:
    performed_ops = [x for x in op]
except InfiniteLoopEntered as e:
    print(e)
else:
    print(op.accumulator)

True
2477
